# First simple web-map with Folium


**Date**: 22.03.2023

**Course**: Methods of Spatial Analysis. Advanced Level. // HSE, Moscow, spring 2023


## 1. Import libraries


In [ ]:

## For working with dataframes

import pandas as pd
import geopandas as gpd
import folium

from shapely import geometry


## 2.Read data from a geojson file


In [ ]:
data = gpd.read_file('./okn_points.geojson')

data_poly = gpd.read_file('./okn_poly.geojson')

data_poly.head()

#### \* clear data and save only valuable attributes

(here is an example how to drop fields, you may need to delete more/less depends, you may choose another way to remove attributes )


In [3]:
data = data.drop(columns=['Тип объекта культурного наследия', 'текстовое описание границ' ])

## 2.Create Grid (fishnet)

- to plot concentration of cultural heritage (or any other point layer) with a choropleth map


#### 2.1. Check CRS of current data and decide do you need to reproject it or not?


- here is an example of reading the current crs and reprojecting geodataframe to another crs (you need to choose by yourself do you need it or not), but I'd rather reccommend to use UTM-zone, when you plot data with folium it is better to choose Web Mercator (EPSG:3857)


In [4]:
print(data.crs.name)

#if you need to reproject you can use .to_crs method
#data = data.to_crs("EPSG:4326")


WGS 84 / UTM zone 37N


#### 2.2. Get the extent of the data


In [5]:
total_bounds = data.total_bounds
minX, minY, maxX, maxY = total_bounds

#### 2.3. Select cell-size


- what are the units?


In [6]:
square_size = 500

#### 2.4. Create fishnet


- you may choose another way to create using QGIS and read prepared data here


In [7]:
grid_cells = []
x, y = (minX, minY)
geom_array = []

while y <= maxY:
        while x <= maxX:
            geom = geometry.Polygon([(x,y), (x, y+square_size), (x+square_size, y+square_size), (x+square_size, y), (x, y)])
            geom_array.append(geom)
            x += square_size
        x = minX
        y += square_size


fishnet = gpd.GeoDataFrame(geom_array, columns=['geometry']).set_crs('EPSG:32637')
fishnet['id'] = fishnet.index

- save final grid to file if you need it


In [8]:
#fishnet.to_file('grid.gpkg')

## 3. Count Points in Polygon


#### 3.1. Merge fishnet and your points data to count them in polygons


In [8]:
merged = gpd.sjoin(data, fishnet, how='left', predicate='within')
merged['n'] = 1
dissolve = merged.dissolve(by="index_right", aggfunc="count")
fishnet.loc[dissolve.index, 'n'] = dissolve.n.values


#### 3.2. Plot the result


- we do it just to check the final result, no any other practical need


In [ ]:
ax = fishnet.plot(column='n', figsize=(12, 8), cmap='viridis', scheme='natural_breaks', k=3, edgecolor="grey")


## 3. Create Web-map


#### 3.1 Map Settings

- carefully look at the function, try to understand what each variable stands for


In [ ]:
data = data.to_crs('EPSG:4326')
m = folium.Map(location=[data.centroid.y.mean(), data.centroid.x.mean()], zoom_start=12,  tiles="cartodb positron", control_scale=True)


#### 3.2 Explore the map box. Do we need any changes?


In [ ]:
m

#### 3.3 Create a Choropleth map based on a fishnet


In [25]:
folium.Choropleth(
    geo_data=fishnet,
    data=fishnet,
    columns=['id', 'n'],
    fill_color='YlGnBu',
    fill_opacity = 0.5,
    key_on='id',
    nan_fill_opacity=0,
   line_color = "#0000",
   legend_name="amount of heritage sites",
   name='Heritage Sites Concentration'
).add_to(m)

- look a the map


In [ ]:
m

#### 3.4 Add heritage buildings with tooltips


- try to color heritage buildings based on their conservation status


In [26]:
#colors = ["orange", "yellow", "green", "blue"]
#categories = data_poly['Категория историко-культурного значения'].unique()

folium.GeoJson(
    data_poly,
    name="Heritage buildings",
    tooltip=folium.GeoJsonTooltip(fields=["r_name"]),
    popup=folium.GeoJsonPopup(fields=['r_name']),
    style_function=lambda x: {
        "fillColor": 'yellow'
    },
    highlight_function=lambda x: {"fillOpacity": 0.8},
    zoom_on_click=True,
    show=False,
).add_to(m)


- look a the map


In [ ]:
m

#### 3.5 Add points cluster


In [14]:
from folium.plugins import MarkerCluster

In [27]:
marker_cluster = MarkerCluster(name='Heritage Sites')
mc1= folium.plugins.FeatureGroupSubGroup(marker_cluster, 'Heritage Sites')
m.add_child(marker_cluster)
m.add_child(mc1)
mc1.add_child(folium.GeoJson(data.to_json(), embed=False, show=False))


In [ ]:
m

#### 3.6 Add different widgets to map


- import plugins


In [28]:
from folium.plugins import MousePosition
from folium.plugins import Fullscreen

In [29]:
folium.LayerControl().add_to(m)

In [30]:
MousePosition().add_to(m)
Fullscreen(
    position="bottomright",
    title="Expand me",
    title_cancel="Exit me",
    force_separate_button=True,
).add_to(m)


In [ ]:
m

## 4. Save map to index.html file and prepare to publish it


In [ ]:
m.save("index.html")